<a href="https://www.kaggle.com/code/franciscangeno/book-recommendation-system?scriptVersionId=114382771" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity
import re
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install gdown
!pip install pandas-gbq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.3/120.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.1/215.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.7/187.7 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.0/177.0 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.4
    Uninstalling protobuf-3.19.4:
      Successfully uninstalled protobuf-3.19.4
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.43.0
    Uninstalling grpcio-1.43.0:
      Successfully uninstalled grpcio-1.43.0
  Attempting uninstall: proto-plus
    Found existing installation: proto-plus 1.20.6
    Uninstalling proto-plus-1.20

In [3]:
from pandas.io import gbq

In [4]:
# get the google drive file
bookdata = !gdown --id 1LXpK1UfqtP89H1tYy0pBGHjYk8IhigUK

In [5]:
import gzip
# we are going to read the file line by line
# streams the file without unzipping it
with gzip.open("/kaggle/working/goodreads_books.json.gz","r") as f:
    line = f.readline()

In [6]:
line
# this is metadata about a single book

b'{"isbn": "0312853122", "text_reviews_count": "1", "series": [], "country_code": "US", "language_code": "", "popular_shelves": [{"count": "3", "name": "to-read"}, {"count": "1", "name": "p"}, {"count": "1", "name": "collection"}, {"count": "1", "name": "w-c-fields"}, {"count": "1", "name": "biography"}], "asin": "", "is_ebook": "false", "average_rating": "4.00", "kindle_asin": "", "similar_books": [], "description": "", "format": "Paperback", "link": "https://www.goodreads.com/book/show/5333265-w-c-fields", "authors": [{"author_id": "604031", "role": ""}], "publisher": "St. Martin\'s Press", "num_pages": "256", "publication_day": "1", "isbn13": "9780312853129", "publication_month": "9", "edition_information": "", "publication_year": "1984", "url": "https://www.goodreads.com/book/show/5333265-w-c-fields", "image_url": "https://images.gr-assets.com/books/1310220028m/5333265.jpg", "book_id": "5333265", "ratings_count": "3", "work_id": "5400751", "title": "W.C. Fields: A Life on Film", "t

In [7]:
# load json line
import json
json.loads(line)
# this returns a dictionary

{'isbn': '0312853122',
 'text_reviews_count': '1',
 'series': [],
 'country_code': 'US',
 'language_code': '',
 'popular_shelves': [{'count': '3', 'name': 'to-read'},
  {'count': '1', 'name': 'p'},
  {'count': '1', 'name': 'collection'},
  {'count': '1', 'name': 'w-c-fields'},
  {'count': '1', 'name': 'biography'}],
 'asin': '',
 'is_ebook': 'false',
 'average_rating': '4.00',
 'kindle_asin': '',
 'similar_books': [],
 'description': '',
 'format': 'Paperback',
 'link': 'https://www.goodreads.com/book/show/5333265-w-c-fields',
 'authors': [{'author_id': '604031', 'role': ''}],
 'publisher': "St. Martin's Press",
 'num_pages': '256',
 'publication_day': '1',
 'isbn13': '9780312853129',
 'publication_month': '9',
 'edition_information': '',
 'publication_year': '1984',
 'url': 'https://www.goodreads.com/book/show/5333265-w-c-fields',
 'image_url': 'https://images.gr-assets.com/books/1310220028m/5333265.jpg',
 'book_id': '5333265',
 'ratings_count': '3',
 'work_id': '5400751',
 'title': '

# **PARSING BOOK METADATA**

In [8]:
# go through a single line in metadata and pick fields we want
def parse_fields(line):
    data=json.loads(line)
    return {
        "book_id": data["book_id"],
        "title":data["title_without_series"],
        "ratings":data["ratings_count"],
        "url":data["url"],
        "cover_image":data["image_url"]       
    }

In [9]:
# go line by line, parsing each line
books_titles = []
with gzip.open("/kaggle/working/goodreads_books.json.gz","r") as f:
    #read every single line
    while True: 
        line = f.readline()
        if not line:
            break
        fields = parse_fields(line)
        try:
# turn ratings into an integer 
# we want to take books that have more than a certain no. of ratings
# we can filter them out as they are not likely to be recommended
            ratings = int(fields["ratings"])
        except ValueError:
            continue
        if ratings > 15:
            books_titles.append(fields) 

# **PROCESSING METADATA WITH PANDAS**


In [10]:
# turn into a dataframe
titles = pd.DataFrame.from_dict(books_titles)

In [11]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [12]:
titles.head(15)

,book_id,title,ratings,url,cover_image
0,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",140,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...
1,6066819,Best Friends Forever,51184,https://www.goodreads.com/book/show/6066819-be...,https://s.gr-assets.com/assets/nophoto/book/11...
2,287141,The Aeneid for Boys and Girls,46,https://www.goodreads.com/book/show/287141.The...,https://s.gr-assets.com/assets/nophoto/book/11...
3,6066812,All's Fairy in Love and War (Avalon: Web of Ma...,98,https://www.goodreads.com/book/show/6066812-al...,https://images.gr-assets.com/books/1316637798m...
4,287149,The Devil's Notebook,986,https://www.goodreads.com/book/show/287149.The...,https://images.gr-assets.com/books/1328768789m...
5,6066814,"Crowner Royal (Crowner John Mystery, #13)",186,https://www.goodreads.com/book/show/6066814-cr...,https://images.gr-assets.com/books/1328724803m...
6,33394837,The House of Memory (Pluto's Snitch #2),269,https://www.goodreads.com/book/show/33394837-t...,https://images.gr-assets.com/books/1493114742m...
7,89373,The Bonfire of the Vanities,77,https://www.goodreads.com/book/show/89373.The_...,https://s.gr-assets.com/assets/nophoto/book/11...
8,89375,90 Minutes in Heaven: A True Story of Death an...,68157,https://www.goodreads.com/book/show/89375.90_M...,https://s.gr-assets.com/assets/nophoto/book/11...
9,89376,Heaven,7345,https://www.goodreads.com/book/show/89376.Heaven,https://images.gr-assets.com/books/1406508230m...


In [13]:
titles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1308957 entries, 0 to 1308956
Data columns (total 5 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   book_id      1308957 non-null  object
 1   title        1308957 non-null  object
 2   ratings      1308957 non-null  object
 3   url          1308957 non-null  object
 4   cover_image  1308957 non-null  object
dtypes: object(5)
memory usage: 49.9+ MB


In [14]:
# ratings column is an object yet should be numeric
titles["ratings"]=pd.to_numeric(titles["ratings"])

In [15]:
titles.describe()

,ratings
count,1.308957e+06
mean,7.279124e+02
std,1.493248e+04
min,1.600000e+01
25%,3.000000e+01
50%,6.400000e+01
75%,1.870000e+02
max,4.899965e+06


In [16]:
titles.isnull().mean()

book_id        0.0
title          0.0
ratings        0.0
url            0.0
cover_image    0.0
dtype: float64

In [17]:
# modify title to minimize search space for the search engine
# replace any characters that don't fall within a certain set lowercase,uppercase,numerals or space
titles["mod_title"]=titles["title"].str.replace("[^a-zA-Z0-9 ]","",regex=True)

In [18]:
titles.head(15)

,book_id,title,ratings,url,cover_image,mod_title
0,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",140,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,The Unschooled Wizard Sun Wolf and Starhawk 12
1,6066819,Best Friends Forever,51184,https://www.goodreads.com/book/show/6066819-be...,https://s.gr-assets.com/assets/nophoto/book/11...,Best Friends Forever
2,287141,The Aeneid for Boys and Girls,46,https://www.goodreads.com/book/show/287141.The...,https://s.gr-assets.com/assets/nophoto/book/11...,The Aeneid for Boys and Girls
3,6066812,All's Fairy in Love and War (Avalon: Web of Ma...,98,https://www.goodreads.com/book/show/6066812-al...,https://images.gr-assets.com/books/1316637798m...,Alls Fairy in Love and War Avalon Web of Magic 8
4,287149,The Devil's Notebook,986,https://www.goodreads.com/book/show/287149.The...,https://images.gr-assets.com/books/1328768789m...,The Devils Notebook
5,6066814,"Crowner Royal (Crowner John Mystery, #13)",186,https://www.goodreads.com/book/show/6066814-cr...,https://images.gr-assets.com/books/1328724803m...,Crowner Royal Crowner John Mystery 13
6,33394837,The House of Memory (Pluto's Snitch #2),269,https://www.goodreads.com/book/show/33394837-t...,https://images.gr-assets.com/books/1493114742m...,The House of Memory Plutos Snitch 2
7,89373,The Bonfire of the Vanities,77,https://www.goodreads.com/book/show/89373.The_...,https://s.gr-assets.com/assets/nophoto/book/11...,The Bonfire of the Vanities
8,89375,90 Minutes in Heaven: A True Story of Death an...,68157,https://www.goodreads.com/book/show/89375.90_M...,https://s.gr-assets.com/assets/nophoto/book/11...,90 Minutes in Heaven A True Story of Death and...
9,89376,Heaven,7345,https://www.goodreads.com/book/show/89376.Heaven,https://images.gr-assets.com/books/1406508230m...,Heaven


In [19]:
# make lowercase title
titles["mod_title"] = titles["mod_title"].str.lower()

In [20]:
# get rid of spaces in a row
# any spaces in a row, replacece with a single space
titles["mod_title"] = titles["mod_title"].str.replace("\s+"," ",regex=True)

In [21]:
titles.head(20)

,book_id,title,ratings,url,cover_image,mod_title
0,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",140,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,the unschooled wizard sun wolf and starhawk 12
1,6066819,Best Friends Forever,51184,https://www.goodreads.com/book/show/6066819-be...,https://s.gr-assets.com/assets/nophoto/book/11...,best friends forever
2,287141,The Aeneid for Boys and Girls,46,https://www.goodreads.com/book/show/287141.The...,https://s.gr-assets.com/assets/nophoto/book/11...,the aeneid for boys and girls
3,6066812,All's Fairy in Love and War (Avalon: Web of Ma...,98,https://www.goodreads.com/book/show/6066812-al...,https://images.gr-assets.com/books/1316637798m...,alls fairy in love and war avalon web of magic 8
4,287149,The Devil's Notebook,986,https://www.goodreads.com/book/show/287149.The...,https://images.gr-assets.com/books/1328768789m...,the devils notebook
5,6066814,"Crowner Royal (Crowner John Mystery, #13)",186,https://www.goodreads.com/book/show/6066814-cr...,https://images.gr-assets.com/books/1328724803m...,crowner royal crowner john mystery 13
6,33394837,The House of Memory (Pluto's Snitch #2),269,https://www.goodreads.com/book/show/33394837-t...,https://images.gr-assets.com/books/1493114742m...,the house of memory plutos snitch 2
7,89373,The Bonfire of the Vanities,77,https://www.goodreads.com/book/show/89373.The_...,https://s.gr-assets.com/assets/nophoto/book/11...,the bonfire of the vanities
8,89375,90 Minutes in Heaven: A True Story of Death an...,68157,https://www.goodreads.com/book/show/89375.90_M...,https://s.gr-assets.com/assets/nophoto/book/11...,90 minutes in heaven a true story of death and...
9,89376,Heaven,7345,https://www.goodreads.com/book/show/89376.Heaven,https://images.gr-assets.com/books/1406508230m...,heaven


In [22]:
# only include titles that have more than 0 length
titles = titles[titles["mod_title"].str.len()>0]

In [23]:
# to json
titles.to_json("books_titles.json")

In [24]:
titles.head(20)


,book_id,title,ratings,url,cover_image,mod_title
0,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",140,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,the unschooled wizard sun wolf and starhawk 12
1,6066819,Best Friends Forever,51184,https://www.goodreads.com/book/show/6066819-be...,https://s.gr-assets.com/assets/nophoto/book/11...,best friends forever
2,287141,The Aeneid for Boys and Girls,46,https://www.goodreads.com/book/show/287141.The...,https://s.gr-assets.com/assets/nophoto/book/11...,the aeneid for boys and girls
3,6066812,All's Fairy in Love and War (Avalon: Web of Ma...,98,https://www.goodreads.com/book/show/6066812-al...,https://images.gr-assets.com/books/1316637798m...,alls fairy in love and war avalon web of magic 8
4,287149,The Devil's Notebook,986,https://www.goodreads.com/book/show/287149.The...,https://images.gr-assets.com/books/1328768789m...,the devils notebook
5,6066814,"Crowner Royal (Crowner John Mystery, #13)",186,https://www.goodreads.com/book/show/6066814-cr...,https://images.gr-assets.com/books/1328724803m...,crowner royal crowner john mystery 13
6,33394837,The House of Memory (Pluto's Snitch #2),269,https://www.goodreads.com/book/show/33394837-t...,https://images.gr-assets.com/books/1493114742m...,the house of memory plutos snitch 2
7,89373,The Bonfire of the Vanities,77,https://www.goodreads.com/book/show/89373.The_...,https://s.gr-assets.com/assets/nophoto/book/11...,the bonfire of the vanities
8,89375,90 Minutes in Heaven: A True Story of Death an...,68157,https://www.goodreads.com/book/show/89375.90_M...,https://s.gr-assets.com/assets/nophoto/book/11...,90 minutes in heaven a true story of death and...
9,89376,Heaven,7345,https://www.goodreads.com/book/show/89376.Heaven,https://images.gr-assets.com/books/1406508230m...,heaven


In [25]:
# create an instance of tfidfvectorizer
# takes a list of strings and turns it into a tfidf matrix. 
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(titles["mod_title"])

In [26]:
# turn search query into a vector then match it up against the matrix
#  then compare
def make_clickable(val):
    return '<a target="_blank" href="{}">Goodreads</a>'.format(val)
def show_image(val):
    return '<img src = "{}" width=50></img>'.format(val)
def search(query, vectorizer):
    # do the same processing that we did with mod_title
    processed = re.sub("[^a-zA-Z0-9 ]", "", query.lower())
    # turn query into a vector
    query_vec = vectorizer.transform([processed])
    # find the similarity
    similarity =cosine_similarity(query_vec,tfidf).flatten()
    indices = np.argpartition(similarity, -10)[-10:]
    results = titles.iloc[indices]
    # the data has a lot of duplicate books so we need to get the row with the highest rating
    results.sort_values("ratings",ascending =False)
    return results.head(5).style.format({'url':make_clickable, 'cover_image':show_image})

In [27]:
# similarity

In [28]:
# # get to the actual book
# results

In [29]:
# search for another book
search("Foundation",vectorizer)

,book_id,title,ratings,url,cover_image,mod_title
694488,9401317,"Foundation (Foundation, #1)",204,Goodreads,,foundation foundation 1
638824,5996629,"Foundation (Foundation, #1)",5359,Goodreads,,foundation foundation 1
1204348,7352028,"Foundation (Foundation, #1)",318,Goodreads,,foundation foundation 1
1229317,414853,"Foundation (Foundation, #1)",604,Goodreads,,foundation foundation 1
66190,16688313,Foundation,43,Goodreads,,foundation


# **CREATE A LIST OF LIKED BOOKS**

In [30]:
# create a list of liked books and store the ids as a string
liked_books = ["8132407","30849411","32737635","5996629"]

# **BOOK RECOMMENDATION DATA**

Using the goodreads interaction file that has; user and how much they liked each book,we are going to create some recommendations 


In [31]:
! head "/kaggle/input/books-id-map/book_id_map.csv"

book_id_csv,book_id
0,34684622
1,34536488
2,34017076
3,71730
4,30422361
5,33503613
6,33517540
7,34467031
8,6383669


In [32]:
# read file in a streaming fashion
# read the file in a line, splitting it on the comma 
# assign the first part to csv_id before the comma and the second part after the comma to book_id
csv_book_mapping = {}
with open("/kaggle/input/books-id-map/book_id_map.csv", "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        csv_id, book_id = line.strip().split(",")
#         in the dictionary, all the keys will be the csv_ids(in the interaction file) and the values are ids in the file we just worked with(metadata)
        csv_book_mapping[csv_id] =book_id      

In [33]:
len(csv_book_mapping)

2360651

In [34]:
interactions = !gdown --id 1zmylV7XW2dfQVCLeg1LbllfQtHD2KUon

In [35]:
! wc -l "/kaggle/working/goodreads_interactions.csv"

228648343 /kaggle/working/goodreads_interactions.csv


# **FIND USERS WITH THE SAME BOOKS AS US**

In [36]:
!head "/kaggle/working/goodreads_interactions.csv"

user_id,book_id,is_read,rating,is_reviewed
0,948,1,5,0
0,947,1,5,1
0,946,1,5,0
0,945,1,5,0
0,944,1,5,0
0,943,1,5,0
0,942,1,5,0
0,941,1,5,0
0,940,1,5,0


In [37]:
overlap_users = set()
with open("/kaggle/working/goodreads_interactions.csv",'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id,_,rating, _, =line.split(",")
        if user_id in overlap_users:
            continue
        try:
            rating = int(rating)
        except ValueError:
            continue
        # turn the csv_id into book_id
        book_id = csv_book_mapping[csv_id]
        if book_id in liked_books and rating >= 4:
            overlap_users.add(user_id)

In [38]:
# find all the books the users read
# loop through the good reads interaction dataset
rec_lines = []
# reclines will only contain books that users who liked the same books as us have read
with open("/kaggle/working/goodreads_interactions.csv",'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        # split the data again
        user_id, csv_id,_,rating, _, =line.split(",")
        if user_id in overlap_users:
            book_id = csv_book_mapping[csv_id]
            rec_lines.append([user_id, book_id, rating])


In [39]:
len(overlap_users)

855

In [40]:
len(rec_lines)

608081

In [41]:
# rank recommmendations in rec_lines
recs=pd.DataFrame(rec_lines, columns = ["user_id","book_id","rating"] )
recs["book_id"]=recs["book_id"].astype(str)

In [42]:
top_recs = recs["book_id"].value_counts().head(10)
# get the values of the index
top_recs=top_recs.index.values

In [43]:
# get from book_id to book title
books_titles = pd.read_json("/kaggle/working/books_titles.json")
books_titles["book_id"]=books_titles["book_id"].astype(str)

In [44]:
books_titles.head()

,book_id,title,ratings,url,cover_image,mod_title
0,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",140,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,the unschooled wizard sun wolf and starhawk 12
1,6066819,Best Friends Forever,51184,https://www.goodreads.com/book/show/6066819-be...,https://s.gr-assets.com/assets/nophoto/book/11...,best friends forever
2,287141,The Aeneid for Boys and Girls,46,https://www.goodreads.com/book/show/287141.The...,https://s.gr-assets.com/assets/nophoto/book/11...,the aeneid for boys and girls
3,6066812,All's Fairy in Love and War (Avalon: Web of Ma...,98,https://www.goodreads.com/book/show/6066812-al...,https://images.gr-assets.com/books/1316637798m...,alls fairy in love and war avalon web of magic 8
4,287149,The Devil's Notebook,986,https://www.goodreads.com/book/show/287149.The...,https://images.gr-assets.com/books/1328768789m...,the devils notebook


In [45]:
# find the book titles where the book id is in these recommendations
books_titles[books_titles["book_id"].isin(top_recs)]

,book_id,title,ratings,url,cover_image,mod_title
284473,2767052,"The Hunger Games (The Hunger Games, #1)",4899965,https://www.goodreads.com/book/show/2767052-th...,https://images.gr-assets.com/books/1447303603m...,the hunger games the hunger games 1
401395,5107,The Catcher in the Rye,2086945,https://www.goodreads.com/book/show/5107.The_C...,https://images.gr-assets.com/books/1398034300m...,the catcher in the rye
436303,32737635,The Most Dangerous Place on Earth,4063,https://www.goodreads.com/book/show/32737635-t...,https://images.gr-assets.com/books/1477347638m...,the most dangerous place on earth
463463,4671,The Great Gatsby,2758812,https://www.goodreads.com/book/show/4671.The_G...,https://images.gr-assets.com/books/1490528560m...,the great gatsby
615314,5470,1984,2023937,https://www.goodreads.com/book/show/5470.1984,https://images.gr-assets.com/books/1348990566m...,1984
790927,2657,To Kill a Mockingbird,3255518,https://www.goodreads.com/book/show/2657.To_Ki...,https://images.gr-assets.com/books/1361975680m...,to kill a mockingbird
833311,6148028,"Catching Fire (The Hunger Games, #2)",1854746,https://www.goodreads.com/book/show/6148028-ca...,https://images.gr-assets.com/books/1358273780m...,catching fire the hunger games 2
878545,3,Harry Potter and the Sorcerer's Stone (Harry P...,4765497,https://www.goodreads.com/book/show/3.Harry_Po...,https://images.gr-assets.com/books/1474154022m...,harry potter and the sorcerers stone harry pot...
1091642,22557272,The Girl on the Train,1076144,https://www.goodreads.com/book/show/22557272-t...,https://images.gr-assets.com/books/1490903702m...,the girl on the train
1095301,11870085,The Fault in Our Stars,2429317,https://www.goodreads.com/book/show/11870085-t...,https://images.gr-assets.com/books/1360206420m...,the fault in our stars


In [46]:
top_recs

array(['2767052', '2657', '4671', '22557272', '11870085', '3', '5470',
       '5107', '32737635', '6148028'], dtype=object)

In [47]:
# value counts on all of the recommendations
all_recs = recs["book_id"].value_counts()

In [48]:
all_recs = all_recs.to_frame().reset_index()

In [49]:
all_recs.head(20)

,index,book_id
0,2767052,430
1,2657,390
2,4671,378
3,22557272,369
4,11870085,364
5,3,363
6,5470,355
7,5107,343
8,32737635,336
9,6148028,331


In [50]:
all_recs.columns = ["book_id","book_count"]

In [51]:
all_recs.head(15)

,book_id,book_count
0,2767052,430
1,2657,390
2,4671,378
3,22557272,369
4,11870085,364
5,3,363
6,5470,355
7,5107,343
8,32737635,336
9,6148028,331


In [52]:
# merge with book titles 
all_recs = all_recs.merge(books_titles,how="inner",on="book_id")

In [53]:
all_recs.head(15)

,book_id,book_count,title,ratings,url,cover_image,mod_title
0,2767052,430,"The Hunger Games (The Hunger Games, #1)",4899965,https://www.goodreads.com/book/show/2767052-th...,https://images.gr-assets.com/books/1447303603m...,the hunger games the hunger games 1
1,2657,390,To Kill a Mockingbird,3255518,https://www.goodreads.com/book/show/2657.To_Ki...,https://images.gr-assets.com/books/1361975680m...,to kill a mockingbird
2,4671,378,The Great Gatsby,2758812,https://www.goodreads.com/book/show/4671.The_G...,https://images.gr-assets.com/books/1490528560m...,the great gatsby
3,22557272,369,The Girl on the Train,1076144,https://www.goodreads.com/book/show/22557272-t...,https://images.gr-assets.com/books/1490903702m...,the girl on the train
4,11870085,364,The Fault in Our Stars,2429317,https://www.goodreads.com/book/show/11870085-t...,https://images.gr-assets.com/books/1360206420m...,the fault in our stars
5,3,363,Harry Potter and the Sorcerer's Stone (Harry P...,4765497,https://www.goodreads.com/book/show/3.Harry_Po...,https://images.gr-assets.com/books/1474154022m...,harry potter and the sorcerers stone harry pot...
6,5470,355,1984,2023937,https://www.goodreads.com/book/show/5470.1984,https://images.gr-assets.com/books/1348990566m...,1984
7,5107,343,The Catcher in the Rye,2086945,https://www.goodreads.com/book/show/5107.The_C...,https://images.gr-assets.com/books/1398034300m...,the catcher in the rye
8,32737635,336,The Most Dangerous Place on Earth,4063,https://www.goodreads.com/book/show/32737635-t...,https://images.gr-assets.com/books/1477347638m...,the most dangerous place on earth
9,6148028,331,"Catching Fire (The Hunger Games, #2)",1854746,https://www.goodreads.com/book/show/6148028-ca...,https://images.gr-assets.com/books/1358273780m...,catching fire the hunger games 2


In [54]:
# create a score to sort the recommendations
all_recs["score"]=all_recs["book_count"] * (all_recs["book_count"]/all_recs["ratings"]) 

In [55]:
all_recs.sort_values("score",ascending=False).head(10)

,book_id,book_count,title,ratings,url,cover_image,mod_title,score
83,30849411,187,Homegoing,559,https://www.goodreads.com/book/show/30849411-h...,https://images.gr-assets.com/books/1473306065m...,homegoing,62.556351
8,32737635,336,The Most Dangerous Place on Earth,4063,https://www.goodreads.com/book/show/32737635-t...,https://images.gr-assets.com/books/1477347638m...,the most dangerous place on earth,27.786365
64,5996629,204,"Foundation (Foundation, #1)",5359,https://www.goodreads.com/book/show/5996629-fo...,https://images.gr-assets.com/books/1301851340m...,foundation foundation 1,7.765628
3014,24909347,24,"Obsidio (The Illuminae Files, #3)",82,https://www.goodreads.com/book/show/24909347-o...,https://images.gr-assets.com/books/1501704611m...,obsidio the illuminae files 3,7.024390
2505,34517748,28,The Beginning of the World in the Middle of th...,113,https://www.goodreads.com/book/show/34517748-t...,https://images.gr-assets.com/books/1500644238m...,the beginning of the world in the middle of th...,6.938053
174,8132407,139,East of Eden,3006,https://www.goodreads.com/book/show/8132407-ea...,https://images.gr-assets.com/books/1328303315m...,east of eden,6.427478
9218,34051963,10,Here We Lie,18,https://www.goodreads.com/book/show/34051963-h...,https://images.gr-assets.com/books/1499097971m...,here we lie,5.555556
8286,36307629,11,"King of Scars (King of Scars, #1)",22,https://www.goodreads.com/book/show/36307629-k...,https://images.gr-assets.com/books/1506962795m...,king of scars king of scars 1,5.500000
3020,28170940,24,"Lethal White (Cormoran Strike, #4)",106,https://www.goodreads.com/book/show/28170940-l...,https://s.gr-assets.com/assets/nophoto/book/11...,lethal white cormoran strike 4,5.433962
8595,32333338,10,Save the Date,19,https://www.goodreads.com/book/show/32333338-s...,https://images.gr-assets.com/books/1510611507m...,save the date,5.263158


In [56]:
# remove some of the books with few ratings
popular_recs = all_recs[all_recs["book_count"]>75].sort_values("score",ascending=False)

In [57]:
def make_clickable(val):
    return '<a target="_blank" href="{}">Goodreads</a>'.format(val)
def show_image(val):
    return '<img src = "{}" width=50></img>'.format(val)
popular_recs[~popular_recs["book_id"].isin(liked_books)].head(15).style.format({'url':make_clickable, 'cover_image':show_image})

,book_id,book_count,title,ratings,url,cover_image,mod_title,score
291,30199414,111,Marlena,5324,Goodreads,,marlena,2.314237
535,5996631,77,"Foundation and Empire (Foundation, #2)",2568,Goodreads,,foundation and empire foundation 2,2.308801
390,30753987,94,The Leavers,5602,Goodreads,,the leavers,1.577294
352,30971707,100,All Grown Up,6505,Goodreads,,all grown up,1.537279
537,29229956,77,Perfect Little World,4113,Goodreads,,perfect little world,1.441527
309,30304221,106,Swimming Lessons,8029,Goodreads,,swimming lessons,1.399427
320,29276588,104,Everything You Want Me to Be,7954,Goodreads,,everything you want me to be,1.359819
199,32796253,131,Final Girls,13141,Goodreads,,final girls,1.305913
113,28815371,167,The Mothers,22346,Goodreads,,the mothers,1.248053
310,28818921,105,Difficult Women,8907,Goodreads,,difficult women,1.237791
